In [10]:
# This script converts Netatmo CSV files to Line Protocol
#
# Line protocol yyntax is 
# <measurement>[,<tag_key>=<tag_value>[,<tag_key>=<tag_value>]] <field_key>=<field_value>[,<field_key>=<field_value>] [<timestamp>]

In [11]:
# CSV file example
#  Name;Long;Lat;ModuleName;ModuleType
#  "Living room";26.004168;44.452849;Outdoor;Outdoor
#  Timestamp;"Timezone : Europe/Bucharest";Temperature;Humidity
#  1467814227;"2016/07/06 17:10:27";29.5;56
#  1467814485;"2016/07/06 17:14:45";29.7;42
#  1467814791;"2016/07/06 17:19:51";29.4;40
#
# Corresponding Line protocol
#  Rosu,lat=44.452849,long=26.004168,ModuleType=Outdoor,Timezone=Europe/Bucharest Temperature=29.5,Humidity=56 1467814227
#  Rosu,lat=44.452849,long=26.004168,ModuleType=Outdoor,Timezone=Europe/Bucharest Temperature=29.7,Humidity=42 1467814485
#  Rosu,lat=44.452849,long=26.004168,ModuleType=Outdoor,Timezone=Europe/Bucharest Temperature=29.4,Humidity=40 1467814791

In [12]:
import pandas as pd
import os
import glob
from datetime import datetime
import pytz

In [13]:
# The field "Measurement" is initialized as the name of the village I live in, which is in the suburban area of Bucharest, Romania
# This has to be specified by the user, as this information in not availlable in the CSV files
# CSV files list only the name I gave to the base station, in muy case "Living room"
measurement = "Rosu"

In [14]:
# Function to convert a row to InfluxDB line protocol
def to_influxdb_line_protocol(row, measurement, tags):
    # This line extracts the timestamp from the row dictionary. It assumes that the row contains a key named 'Timestamp'
    timestamp = row['Timestamp']
    # This line constructs the field set for the InfluxDB line protocol. 
    # It extracts the Temperature and Humidity values from the row dictionary and formats them into a string.
    # f"Temperature={row['Temperature']},Humidity={row['Humidity']}": This is an f-string (formatted string literal) that embeds the values of Temperature and Humidity into the string.
    fields = f"Temperature={row['Temperature']},Humidity={row['Humidity']}"
    # This line constructs the tag set for the InfluxDB line protocol.
    # tags.items(): This returns a view object that displays a list of a dictionary's key-value tuple pairs.
    # [f"{key}={value}" for key, value in tags.items()]: This is a list comprehension that iterates over the key-value pairs in the tags dictionary and formats each pair as key=value.
    # ",".join(...): This joins the list of formatted tag strings into a single string, with each pair separated by a comma
    tag_set = ",".join([f"{key}={value}" for key, value in tags.items()])
    # This line combines the measurement name, tag set, field set, and timestamp into the InfluxDB line protocol format.
    # f"{measurement},{tag_set} {fields} {timestamp}": This is another f-string that formats the final line protocol string 
    # with the measurement, tag set, fields, and timestamp separated by appropriate delimiters.
    return f"{measurement},{tag_set} {fields} {timestamp}"

In [15]:
# Get all CSV files in the current directory
# The glob module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell. It is useful for searching for files with specific patterns in their names.
# glob.glob("*.csv"): This function call searches for all files in the current directory that have a .csv extension. The "*.csv" is a pattern where * matches any number of characters. T
# csv_files: This variable will be a list containing the filenames of all the .csv files found in the current directory
csv_files = glob.glob("*.csv")
# This line prints the list of .csv files found. If no .csv files are found, it will print an empty list []
print(csv_files)

['debug_2024.csv', 'Outdoor_6_24_2024(7).csv', 'Outdoor_6_24_2024(30).csv', 'Outdoor_6_24_2024(24).csv', 'Outdoor_6_24_2024(4).csv', 'Outdoor_6_24_2024(19).csv', 'lp_2016_10.csv', 'lp_2020_07.csv', 'lp_2018_07.csv', 'Outdoor_6_24_2024(12).csv', 'Outdoor_6_24_2024(22).csv', 'Outdoor_6_24_2024(14).csv', 'Outdoor_6_24_2024(21).csv', 'Outdoor_6_24_2024(2).csv', 'lp_2022_01.csv', 'Outdoor_6_24_2024(28).csv', 'Outdoor_6_24_2024(15).csv', 'lp_2017_04.csv', 'lp_2023_01.csv', 'lp_2017_10.csv', 'Outdoor_6_24_2024(25).csv', 'Outdoor_6_24_2024(26).csv', 'lp_2020_04.csv', 'lp_2018_04.csv', 'Outdoor_6_24_2024(16).csv', 'lp_2017_07.csv', 'Outdoor_6_24_2024(10).csv', 'lp_2019_04.csv', 'lp_2018_01.csv', 'Outdoor_6_24_2024(5).csv', 'lp_2023_10.csv', 'Outdoor_6_24_2024(20).csv', 'lp_2023_04.csv', 'Outdoor_6_24_2024(1).csv', 'lp_2021_10.csv', 'Outdoor_6_24_2024(3).csv', 'lp_2020_10.csv', 'lp_2024_01.csv', 'Outdoor_6_24_2024(31).csv', 'lp_2024_06.csv', 'lp_2019_10.csv', 'Outdoor_6_24_2024.csv', 'lp_2016_07

In [16]:
# Process CSV Files
for input_file in csv_files:
    with open(input_file, 'r') as file:
        first_line = file.readline().strip()

    # Check if the first line contains the required header
    if first_line != "Name;Long;Lat;ModuleName;ModuleType":
        print(f"File {input_file} does not have the required header. Skipping.")
        continue

    # Read the metadata (first two rows)
    metadata = pd.read_csv(input_file, sep=';', nrows=2)

    # Metadata example (rows 0:1)
    # Name;Long;Lat;ModuleName;ModuleType
    # "Living room";26.004168;44.452849;Outdoor;Outdoor

    # Remove double quotes from metadata columns
    for col in metadata.columns:
        if metadata[col].dtype == 'object':
            metadata[col] = metadata[col].map(lambda x: x.strip('"') if isinstance(x, str) else x)

    # Extract latitude, longitude, ModuleType, and Timezone from metadata
    lat = metadata['Lat'][0]
    long = metadata['Long'][0]
    module_type = metadata['ModuleType'][0]

    # Data example (rows 2:...)
    # Timestamp;"Timezone : Europe/Bucharest";Temperature;Humidity
    # 1467814227;"2016/07/06 17:10:27";29.5;56

    # Timezone is in the second row, second column
    # metadata.iloc[1, 1]: This accesses the element in the second row and second column of the DataFrame. Indexing in pandas starts from 0, so 1 refers to the second row and second column.
    # .split(':')[1]: This splits the string at the colon (:) and selects the second part (index 1). Assuming the string is formatted as "key: timezone", this will extract the timezone part.
    timezone_str = metadata.iloc[1, 1].split(':')[1].strip()
    # The purpose of this line of code is to convert a string representation of a timezone into a pytz timezone object. This object can then be used for timezone-aware date and time operations.
    timezone = pytz.timezone(timezone_str)

    # Read the time-series data starting from the correct row
    time_series_data = pd.read_csv(input_file, sep=';', skiprows=2)

    # Remove double quotes from time-series data columns
    for col in time_series_data.columns:
        if time_series_data[col].dtype == 'object':
            time_series_data[col] = time_series_data[col].map(lambda x: x.strip('"') if isinstance(x, str) else x)

    # Strip any leading/trailing spaces from column names
    time_series_data.columns = time_series_data.columns.str.strip()

    # Extract the date from the first measurement
    # This will be used for output file naming
    # Is needed because Unix timestamps are UTC
    # and I want the files to be named according to the local time
    first_timestamp = int(time_series_data['Timestamp'].iloc[0])
    first_date_utc = datetime.utcfromtimestamp(first_timestamp).replace(tzinfo=pytz.utc)
    first_date_local = first_date_utc.astimezone(timezone)
    year_month = first_date_local.strftime('%Y_%m')  # Format to "YYYY_MM"

    # Print debug information
    print(f"File: {input_file}")
    print(f"First timestamp: {first_timestamp}")
    print(f"First date (UTC): {first_date_utc}")
    print(f"First date (Local): {first_date_local}")
    print(f"Year and month: {year_month}")

    # Convert time-series data to InfluxDB line protocol
    tags = {"lat": lat, "long": long, "ModuleType": module_type, "Timezone": timezone_str}

    lines = time_series_data.apply(lambda row: to_influxdb_line_protocol(row, measurement, tags), axis=1)

    # Create new filename based on the date of the first measurement
    output_file = f"{measurement}_{year_month}.lp"

    # Save the result to a new CSV file without quotes
    with open(output_file, 'w') as f:
        for line in lines:
            f.write(line + '\n')

    print(f"InfluxDB line protocol data has been saved to {output_file}")

File debug_2024.csv does not have the required header. Skipping.
File: Outdoor_6_24_2024(7).csv
First timestamp: 1522530223
First date (UTC): 2018-03-31 21:03:43+00:00
First date (Local): 2018-04-01 00:03:43+03:00
Year and month: 2018_04
InfluxDB line protocol data has been saved to Rosu_2018_04.lp
File: Outdoor_6_24_2024(30).csv
First timestamp: 1704060279
First date (UTC): 2023-12-31 22:04:39+00:00
First date (Local): 2024-01-01 00:04:39+02:00
Year and month: 2024_01
InfluxDB line protocol data has been saved to Rosu_2024_01.lp
File: Outdoor_6_24_2024(24).csv
First timestamp: 1656623010
First date (UTC): 2022-06-30 21:03:30+00:00
First date (Local): 2022-07-01 00:03:30+03:00
Year and month: 2022_07
InfluxDB line protocol data has been saved to Rosu_2022_07.lp
File: Outdoor_6_24_2024(4).csv
First timestamp: 1498856608
First date (UTC): 2017-06-30 21:03:28+00:00
First date (Local): 2017-07-01 00:03:28+03:00
Year and month: 2017_07
InfluxDB line protocol data has been saved to Rosu_2017